In [1]:
'''
s0_0_img_retrival
检索指定目录下所有指定文件(图片),保存到txt中
'''
from glob import glob
import os

# 定义
file_type = 'jpg'
retrival_path = '/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/train_clip_pic_16'
output_txt = '/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/train_clip_pic_16.txt'

# glob检索
file_list = glob(f"{retrival_path}/*.{file_type}")

# 保存到txt中
with open(output_txt, 'a') as f:
    for file in file_list:
        f.write(file + '\n')


In [ ]:
'''
s0_1_split_json
将大的json文件拆分成单个对应json文件
'''
import json
import os

# input_json = '/home/hf/code/liuzhiwei/AIGC/LMM/caption1/test.json'
input_json = '/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/73_train.json'
output_json_path = '/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/train_clip_pic_16'
# 打开 JSON 文件
with open(input_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 遍历 JSON 文件中的每个字典
for i, dic in enumerate(data):
    # 获取字典中的 'image' 属性，假设它是一个图片的 URL
    img_url = dic['image']
    # 从 URL 中获取图片的文件名，假设它是以 .jpg 结尾的
    img_name = img_url.split('/')[-1]
    img_name_prefix = img_name.split('.')[0]

    # 将字典转换为 JSON 格式，并保存在新的文件夹中，以 i.json 为文件名
    output_json = os.path.join(output_json_path, f'{img_name_prefix}.json')
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(dic, f, ensure_ascii=False, indent=4)

In [8]:
%%bash
# '''
# s0_2_split_img
# 将图片分成n份以便进行多进程推理(sh文件)
# '''

# funcs
doMakeDir(){
    path=$1
    if [ ! -d $path ]
    then 
        mkdir -p $path
    fi
}
# 指定pwd
path='/home/hf/code/liuzhiwei/AIGC/LMM/caption1/split_file' 
cd $path
name='train_clip_pic_16'
doMakeDir "${path}/${name}"
cd "${name}"

# 定义输入txt
input_txt="${name}.txt"
# 拷贝到当前目录
cp "/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/${input_txt}" ./
# 获取总长度
total_lens=$(wc -l < $input_txt) 
# 分成n份
len_per_file=$(echo "$total_lens / 10" | bc)
# split -d指定输出前缀 -a指定输出后缀输出长度 -l指定输入文件
split -d -a 2 -l $len_per_file $input_txt slice00

In [ ]:
'''
s1_info_copy
单进程简单复制json无需变动信息
'''

import os
import json
from PIL import Image

# input_txt
input_txt = f'/home/hf/code/liuzhiwei/AIGC/LMM/caption1/split_file/train_clip_pic_16/train_clip_pic_16.txt'

# func
def get_img_reso(img_path):
    with Image.open(img_path) as img:
        return img.size

# copy info
if os.path.exists(input_txt):
    with open(input_txt, 'r') as f:
        i = 0
        content = {}
        copy_keys = ['image', 'caption', 'translation', 'relationship']
        for line in f:
            line = line.strip()
            line_prefix, line_postfix = os.path.splitext(line)
            # get img_reso
            width, height = get_img_reso(line)

            # get input_json/output_json
            input_json = line_prefix + '.json'
            output_json = line_prefix + '_output.json'

            # get input_json's content
            if os.path.exists(input_json):
                with open(input_json, 'r') as f:
                    data = json.load(f)
                    for key in copy_keys:
                        if key in data:
                            content[key] = data[key]
                        else:
                            print(f'Warning: Key {key} not found in {input_json}')

                # add other keys that do not need to be changed either
                content['id'] = i
                content['annotator'] = 2
                content['annotator_id'] = i
                content['created_at'] = "2024-01-07T03:01:10.604652Z"
                content['updated_at'] = "2024-01-07T03:01:10.604652Z"
                content['lead_time'] = 348.555
                content['Label'] = [
                    {
                        "x": 0,
                        "y": 0,
                        "width": 10,
                        "height": 10,
                        "rotation": 0,
                        "rectanglelabels": [
                            "Daytime"
                        ],
                        "original_width": width,
                        "original_height": height
                    }
                ]
                i += 1

                # copy to output_json
                with open(output_json, 'w') as f:
                    json.dump(content, f, indent=4, ensure_ascii=False)
            else:
                print(f'Error: File {input_json} not found')
else:
    print(f'Error: File {input_txt} not found')


In [5]:
'''
s3_merge_json
合并所有json
'''
import os
import json
import glob

# definition
input_path = f'/home/hf/code/liuzhiwei/AIGC/LMM/caption1/origin_input/train_clip_pic_16'
output_path = f'/home/hf/code/liuzhiwei/AIGC/LMM/caption1/json_dir/train_clip_pic_16/output_final.json'
files = glob.glob(os.path.join(input_path, '*output.json'))  # Remove the extra slash here
merged_data = []

# traversal
for file in files:
    # 读取 JSON 文件
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # 合并数据
    merged_data.append(data)

# save to final json
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)